In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load clean data

df=pd.read_csv('data/clean_data.csv')

In [3]:
pd.options.display.max_columns=100   #to see all columns

df

,Unnamed: 0,status_group,funder,gps_height,installer,longitude,latitude,basin,region,district_code,lga,ward,population,public_meeting,permit,construction_year,extraction_type_group,management,payment,water_quality,quantity,source,waterpoint_type,decade
0,0,0,Roman,1390,Roman,34.938093,-9.856322,Lake Nyasa,Iringa,5,Ludewa,Mundindi,109,True,False,1999,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,90s
1,1,0,Grumeti,1399,GRUMETI,34.698766,-2.147466,Lake Victoria,Mara,2,Serengeti,Natta,280,True,True,2010,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,10s
2,2,0,Lottery Club,686,world vision,37.460664,-3.821329,Pangani,Manyara,4,Simanjiro,Ngorika,250,True,True,2009,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,00s
3,3,2,Unicef,263,Unicef,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,Nanyumbu,58,True,True,1986,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,80s
4,4,0,Action In A,0,Artisan,31.130847,-1.825359,Lake Victoria,Kagera,1,Karagwe,Nyakasimbi,281,True,True,2000,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,59395,0,Germany Republi,1210,CES,37.169807,-3.253847,Pangani,Kilimanjaro,5,Hai,Masama Magharibi,125,True,True,1999,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe,90s
59396,59396,0,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Rufiji,Iringa,4,Njombe,Ikondo,56,True,True,1996,gravity,vwc,pay annually,soft,enough,river,communal standpipe,90s
59397,59397,0,Unknown,0,Unknown,34.017087,-8.750434,Rufiji,Mbeya,7,Mbarali,Chimala,281,True,False,2000,swn 80,vwc,pay monthly,fluoride,enough,machine dbh,hand pump,0
59398,59398,0,Malec,0,Musa,35.861315,-6.378573,Rufiji,Dodoma,4,Chamwino,Mvumi Makulu,281,True,True,2000,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,0


In [4]:
# dropping unnamed column and other unnecessary columns
df.drop(columns=['Unnamed: 0', 'funder', 'installer', 'construction_year'], inplace=True)

In [5]:
df.head()

,status_group,gps_height,longitude,latitude,basin,region,district_code,lga,ward,population,public_meeting,permit,extraction_type_group,management,payment,water_quality,quantity,source,waterpoint_type,decade
0,0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,5,Ludewa,Mundindi,109,True,False,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,90s
1,0,1399,34.698766,-2.147466,Lake Victoria,Mara,2,Serengeti,Natta,280,True,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,10s
2,0,686,37.460664,-3.821329,Pangani,Manyara,4,Simanjiro,Ngorika,250,True,True,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,00s
3,2,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,Nanyumbu,58,True,True,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,80s
4,0,0,31.130847,-1.825359,Lake Victoria,Kagera,1,Karagwe,Nyakasimbi,281,True,True,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,0


We will first try to simplify our problem to find the best model. We will use different encoder, scaler, classifier and try different models.For simplicity, we will merge 'functional' and 'functional but needs repair' and assign this to 1. We will assign 'non-functional' to 0. 
We will use a simple model then improve on it. After finding the best model, we will try it for the multi-class label format.

In [6]:
# importing all necessary libraries

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer,RobustScaler
from sklearn.model_selection import cross_val_score
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, GridSearchCV

import warnings
warnings.filterwarnings("ignore")

import category_encoders as ce
from category_encoders import WOEEncoder
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder, LeaveOneOutEncoder, JamesSteinEncoder, MEstimateEncoder

from mlxtend.evaluate import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import MinMaxScaler
from mlxtend.evaluate import feature_importance_permutation
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier

from sklearn.svm import SVC
import gc; gc.enable()

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.neighbors import KNeighborsClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [7]:
# TO protect the original clead data, we will make a copy of it

df1 = df.copy()

In [8]:
df1

,status_group,gps_height,longitude,latitude,basin,region,district_code,lga,ward,population,public_meeting,permit,extraction_type_group,management,payment,water_quality,quantity,source,waterpoint_type,decade
0,0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,5,Ludewa,Mundindi,109,True,False,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,90s
1,0,1399,34.698766,-2.147466,Lake Victoria,Mara,2,Serengeti,Natta,280,True,True,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,10s
2,0,686,37.460664,-3.821329,Pangani,Manyara,4,Simanjiro,Ngorika,250,True,True,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,00s
3,2,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,Nanyumbu,Nanyumbu,58,True,True,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,80s
4,0,0,31.130847,-1.825359,Lake Victoria,Kagera,1,Karagwe,Nyakasimbi,281,True,True,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,0,1210,37.169807,-3.253847,Pangani,Kilimanjaro,5,Hai,Masama Magharibi,125,True,True,gravity,water board,pay per bucket,soft,enough,spring,communal standpipe,90s
59396,0,1212,35.249991,-9.070629,Rufiji,Iringa,4,Njombe,Ikondo,56,True,True,gravity,vwc,pay annually,soft,enough,river,communal standpipe,90s
59397,0,0,34.017087,-8.750434,Rufiji,Mbeya,7,Mbarali,Chimala,281,True,False,swn 80,vwc,pay monthly,fluoride,enough,machine dbh,hand pump,0
59398,0,0,35.861315,-6.378573,Rufiji,Dodoma,4,Chamwino,Mvumi Makulu,281,True,True,nira/tanira,vwc,never pay,soft,insufficient,shallow well,hand pump,0


In [9]:
# dropping the Iga and Ward columns

df1.drop(columns=['lga', 'ward'], inplace=True)

In [10]:
# Changing the permit and public_meeting columns with boolean values to integers/binary

df1['permit'] = df1['permit'].astype(bool).astype(int)
df1['public_meeting'] = df1['public_meeting'].astype(bool).astype(int)

In [11]:
df1.head()

,status_group,gps_height,longitude,latitude,basin,region,district_code,population,public_meeting,permit,extraction_type_group,management,payment,water_quality,quantity,source,waterpoint_type,decade
0,0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,5,109,1,0,gravity,vwc,pay annually,soft,enough,spring,communal standpipe,90s
1,0,1399,34.698766,-2.147466,Lake Victoria,Mara,2,280,1,1,gravity,wug,never pay,soft,insufficient,rainwater harvesting,communal standpipe,10s
2,0,686,37.460664,-3.821329,Pangani,Manyara,4,250,1,1,gravity,vwc,pay per bucket,soft,enough,dam,communal standpipe multiple,00s
3,2,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,63,58,1,1,submersible,vwc,never pay,soft,dry,machine dbh,communal standpipe multiple,80s
4,0,0,31.130847,-1.825359,Lake Victoria,Kagera,1,281,1,1,gravity,other,never pay,soft,seasonal,rainwater harvesting,communal standpipe,0


In [12]:
df1.columns

Index(['status_group', 'gps_height', 'longitude', 'latitude', 'basin',
       'region', 'district_code', 'population', 'public_meeting', 'permit',
       'extraction_type_group', 'management', 'payment', 'water_quality',
       'quantity', 'source', 'waterpoint_type', 'decade'],
      dtype='object')

Next, we will use scaler for numeric columns and encoder for categorical 
columns. To do this, we will divide the columns into two:

In [13]:
# categorical columns

cat_col = ['basin','region','extraction_type_group','management','payment','water_quality','quantity',
               'source','waterpoint_type','decade','installer_cat','funder_cat']

In [14]:
# numerical columns

num_col = ['gps_height','longitude','latitude','district_code','population','public_meeting','permit']

In [15]:
df1['status_group'].value_counts()

status_group
0    32259
2    22824
1     4317
Name: count, dtype: int64

0: functional
1: functional but needs repair
2: non-fucntional


As explained before, we will take 'functional' and 'functional but needs repair' together and make them 1, non-functional will be represented by 0.

In [16]:
# Getting together the labe;s and converting them

new_status_group = {0:1, 1:1, 2:0}
df1['status_group'] = df1['status_group'].replace(new_status_group)

In [17]:
df1['status_group'].value_counts()

status_group
1    36576
0    22824
Name: count, dtype: int64

1: functional

0: no-functional

Now we will assign our target column

In [18]:
target = 'status_group'

**Pipeline / Baseline**

In [19]:
# Dividing X and target

variables = [c for c in df1.columns.tolist() if c not in [target]]
X = df1[variables]
y = df1[target]

In [20]:
# Dviding the X and y to test and train

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

We will first use the train test split to figure out our problem. After learning from the baseline, we will use cross validation technique to find the best result, because it is more convenient and easy to understand what is going on.
For some models we will use both of them to check if our results are consistent or not.

In [21]:
# Creating an empty dataframe to keep track of our results.

df_results = pd.DataFrame(columns=["model", "scaler", "encoder"])

**Baseline - Robust Scaler / Target Encoder with LogReg**

To scale numeric values and encode categorical columns, we will make a pipeline and also use it in our model and classifier changes. For the first trial we will use Robust Scaler as a scaler. Robust scales variables using statistics that are strong to outliers. Robust Scaler uses IQR(Interquartile Range). For an encoder, we will try target encoder which works well with higher cardinality features and our data has higher unique values also. Our first trial for baseline is Logistic Regression which predicts the probability that a certain instance belongs to a class. We will use balanced as class weight, because our classes are imbalanced.

In [22]:
# making pipeline

scaler = RobustScaler()
encoder = ce.TargetEncoder(cols=cat_col)

# Putting numeric columns to scaler and categorical to encoder
num_transform = make_pipeline(scaler)
cat_transform = make_pipeline(encoder)

# getting together our scaler and encoder with preprocessor
preprocessor = ColumnTransformer(transformers=[('num', num_transform, num_col),('cat', cat_transform, cat_col)])

# choosing model
lr =  LogisticRegression(class_weight='balanced', solver='1bfgs', random_state=42)

# giving all values to pipeline
pipe = make_pipeline(preprocessor,lr)
pipe.fit(X_train, y_train)

# make predictions on the training set
y_pred = pipe.predict(X_train)

# make predictions on test set
y_pred_test = pipe.predict(X_test)

# to print the results in good way
print("Accuracy:"); print("="*len("Accuracy:"))
print(f"TRAIN: {accuracy_score(y_train, y_pred)}")
print(f"TEST: {accuracy_score(y_test, y_pred_test)}")

print("\nBalanced Accuracy:"); print("="*len("Balanced Accuracy:"))
print(f"TRAIN: {balanced_accuracy_score(y_train, y_pred)}")
print(f"TEST: {balanced_accuracy_score(y_test, y_pred_test)}")

ValueError: A given column is not a column of the dataframe